In [1]:
import os
import json
import pandas as pd
from functools import reduce

In [2]:
def unpack_dict(data):
    L = []
    for x in data:
        d = {}
        for k,v in x.items():
            if k == 'values':
                for k2,v2 in v.items():
                    d[k2] = v2
            else:
                d[k] = v    
        L.append(d)
    return L

if __name__ == '__main__':
    path = os.getcwd()
    frames =[]
    for files in os.listdir(path):
        if '.json' in files:
            with open(path + '/' + files) as json_data:
                data = json.load(json_data)
                if any(x in files for x in ['energy', 'co2']):
                    df = pd.DataFrame(data)
                    frames.append(df)
                else:
                    df = pd.DataFrame(unpack_dict(data))
                    col_names = [col_name for col_name in df.columns if col_name not in ['name', 'units', 'year']]
                    df[col_names] = df[col_names].apply(pd.to_numeric, errors='coerce')
                    new_col_name = f'{files.split("_")[0]}_{df.filter(regex="(deficit$)",axis=1).columns[0]}/surplus'
                    deficit = df.filter(regex='(deficit$)',axis=1).columns[0]
                    surplus = df.filter(regex='(surplus$)+',axis=1).columns[0]
                    df[new_col_name] = df[[deficit, surplus]].sum(axis=1)
                    df.drop([deficit, surplus], axis = 1, inplace=True)
                    df = df.rename(columns = { col : files.split("_")[0] + '_' + col for col in ['net_imports', 'year', 'units', 'net_exports']})
                    frames.append(df)

    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['name'],
                                                how='outer'), frames)


In [3]:
column_names = ['name', 'energy_consumption_btu', 'world_share','pc_yearly_btu', 'non_renewable', 'oil_consump', 'gas_consump','coal_consump', 're_nuclear', 'co2_emissions_tons_2016', 'co2_emiss_one_year_change', 'population_2016', 'co2_emiss_per_capita','country_share_of_world_co2','coal_reserves', 'coal_production', 'coal_consumption',
       'coal_imports', 'coal_exports', 'coal_net_imports', 'coal_year',
       'coal_units', 'coal_net_exports', 'coal_yearly_deficit/surplus',
       'gas_reserves', 'gas_production', 'gas_consumption', 'gas_imports',
       'gas_exports', 'gas_net_imports', 'gas_year', 'gas_units',
       'gas_net_exports', 'gas_yearly_deficit/surplus', 'oil_reserves',
       'oil_production', 'oil_consumption', 'oil_imports', 'oil_exports',
       'oil_net_imports', 'oil_year', 'oil_units', 'oil_net_exports',
       'oil_daily_deficit/surplus']

In [4]:
df_merged[column_names]

,name,energy_consumption_btu,world_share,pc_yearly_btu,non_renewable,oil_consump,gas_consump,coal_consump,re_nuclear,co2_emissions_tons_2016,...,oil_reserves,oil_production,oil_consumption,oil_imports,oil_exports,oil_net_imports,oil_year,oil_units,oil_net_exports,oil_daily_deficit/surplus
0,United Arab Emirates,4667004234000,0.80138%,491926,100%,39%,59%,2%,0%,218788684,...,9.780000e+10,3772788.0,896000.0,0.0,2487580.0,NaN,2016,Barrels,2487580.0,2876788.0
1,Ukraine,3632194307000,0.62369%,81645,73%,13%,30%,29%,27%,233220080,...,3.950000e+08,55832.0,244000.0,12548.0,0.0,12548.0,2016,Barrels,NaN,-188168.0
2,Kazakhstan,3574850089000,0.61385%,197724,97%,18%,16%,64%,3%,231919540,...,3.000000e+10,1698280.0,325000.0,469.0,1245575.0,NaN,2016,Barrels,1245106.0,1373280.0
3,Singapore,3630377394000,0.62338%,636011,100%,86%,13%,0%,0%,48381759,...,0.000000e+00,42274.0,1357000.0,1056958.0,27033.0,1029925.0,2016,Barrels,NaN,-1314726.0
4,Egypt,4012356526000,0.68897%,41604,96%,41%,53%,2%,4%,219377350,...,4.400000e+09,682904.0,877000.0,70411.0,240295.0,NaN,2016,Barrels,169884.0,-194096.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Antigua and Barbuda,10605278000,0.00182%,111136,97%,97%,0%,0%,3%,438763,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,South Sudan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4038496,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,Palau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,Anguilla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_merged[column_names].to_csv('worldometers_energy.csv', index=False)